In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os

In [49]:
current_dir = os.getcwd() # Obtener la ruta del directorio actual del notebook
ROOT_PATH = os.path.dirname(current_dir) # Obtener la ruta del directorio superior
sys.path.insert(1, ROOT_PATH) # Insertar la ruta en sys.path
import root 

In [50]:
#creating paths from root 
client_path = root.DIR_DATA_STAGE + '1_single/client.pkl'
electricity_prices_path = root.DIR_DATA_STAGE + '1_single/electricity_prices.pkl'
gas_prices_path = root.DIR_DATA_STAGE + '1_single/gas_prices.pkl'
historical_weather_path = root.DIR_DATA_STAGE + '1_single/historical_weather.pkl'
train_path_c = root.DIR_DATA_STAGE + '1_single/train_consumption.pkl'
train_path_p = root.DIR_DATA_STAGE + '1_single/train_production.pkl'

In [51]:
# Read datasets from local
client = pd.read_pickle(client_path)
electricity_prices = pd.read_pickle(electricity_prices_path)
gas_prices = pd.read_pickle(gas_prices_path)
historical_weather = pd.read_pickle(historical_weather_path)
train_c = pd.read_pickle(train_path_c)
train_p = pd.read_pickle(train_path_p)

In [52]:
train_p.isnull().sum()

target      0
datetime    0
dtype: int64

In [53]:
train_p['date'] = train_p['datetime'].dt.floor('D')
train_p.head()

,target,datetime,date
10,0.315,2021-09-01 00:00:00,2021-09-01
132,0.353,2021-09-01 01:00:00,2021-09-01
254,0.338,2021-09-01 02:00:00,2021-09-01
376,0.303,2021-09-01 03:00:00,2021-09-01
498,0.424,2021-09-01 04:00:00,2021-09-01


In [54]:
train_p.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15312 entries, 10 to 2018234
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   target    15312 non-null  float64       
 1   datetime  15312 non-null  datetime64[ns]
 2   date      15312 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(1)
memory usage: 478.5 KB


In [55]:
gas_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   forecast_date          637 non-null    datetime64[ns]
 1   lowest_price_per_mwh   637 non-null    float64       
 2   highest_price_per_mwh  637 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 15.1 KB


In [56]:
train_c['date'] = train_c['datetime'].dt.floor('D')
train_c.head()

,target,datetime,date
11,5155.056,2021-09-01 00:00:00,2021-09-01
133,4953.258,2021-09-01 01:00:00,2021-09-01
255,4910.692,2021-09-01 02:00:00,2021-09-01
377,4700.725,2021-09-01 03:00:00,2021-09-01
499,4857.681,2021-09-01 04:00:00,2021-09-01


In [57]:
client['date_2_join'] = pd.to_datetime(client['date']) + pd.Timedelta(days=3)
client.head()

,eic_count,installed_capacity,date,date_2_join
5,265,13417.79,2021-09-01,2021-09-04
66,265,13417.79,2021-09-02,2021-09-05
127,265,13417.79,2021-09-03,2021-09-06
188,265,13417.79,2021-09-04,2021-09-07
249,265,13417.79,2021-09-05,2021-09-08


In [58]:
electricity_prices['date_2_join'] = pd.to_datetime(electricity_prices['forecast_date']) + pd.Timedelta(days=1)
electricity_prices.drop(columns=['forecast_date'], inplace=True)
electricity_prices.head()

,euros_per_mwh,date_2_join
0,92.51,2021-09-02 00:00:00
1,88.90,2021-09-02 01:00:00
2,87.35,2021-09-02 02:00:00
3,86.88,2021-09-02 03:00:00
4,88.43,2021-09-02 04:00:00


In [59]:
gas_prices['date_2_join'] = pd.to_datetime(gas_prices['forecast_date']) + pd.Timedelta(days=1)
gas_prices.drop(columns=['forecast_date'], inplace=True)
gas_prices.head()

,lowest_price_per_mwh,highest_price_per_mwh,date_2_join
0,45.23,46.32,2021-09-02
1,45.62,46.29,2021-09-03
2,45.85,46.40,2021-09-04
3,46.30,46.80,2021-09-05
4,46.30,46.58,2021-09-06


In [31]:
historical_weather['date_2_join'] = historical_weather['datetime'] + pd.Timedelta(days=2)
historical_weather.drop(columns=['datetime'], inplace=True)
historical_weather.head()

,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,date_2_join
0,13.79,10.73,0.0,0.0,1010.56,54.4,56.9,3.5,3.9,5.325000,342.0,0.0,0.0,0.0,2021-09-03 00:00:00
1,13.62,10.35,0.0,0.0,1010.47,28.9,26.0,1.1,15.8,5.327778,345.8,0.0,0.0,0.0,2021-09-03 01:00:00
2,13.13,10.07,0.0,0.0,1010.14,13.7,14.5,0.1,1.8,5.091667,340.8,0.0,0.0,0.0,2021-09-03 02:00:00
3,12.74,9.94,0.0,0.0,1010.09,8.6,6.5,0.9,7.5,4.944444,335.5,0.0,0.0,0.0,2021-09-03 03:00:00
4,12.46,10.03,0.0,0.0,1009.83,4.1,2.8,0.6,4.2,4.669444,330.6,0.0,0.0,0.0,2021-09-03 04:00:00


## Mergin data sets

In [32]:
len(train_c)

15312

In [33]:
len(train_p)

15312

In [34]:
len(client)

636

## Merged consumption

In [35]:
client.head()

,eic_count,installed_capacity,date,date_2_join
5,265,13417.79,2021-09-01,2021-09-04
66,265,13417.79,2021-09-02,2021-09-05
127,265,13417.79,2021-09-03,2021-09-06
188,265,13417.79,2021-09-04,2021-09-07
249,265,13417.79,2021-09-05,2021-09-08


In [36]:
merged_df = train_c.merge(historical_weather, left_on=['datetime',], right_on=['date_2_join',], how='left')
merged_df.drop(columns=['date_2_join'], inplace=True)
merged_df = merged_df.merge(gas_prices, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df.drop(columns=['date_2_join'], inplace=True)
merged_df = merged_df.merge(electricity_prices, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df.drop(columns=['date_2_join'], inplace=True)
merged_df = merged_df.merge(client, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df.drop(columns=['date_2_join'], inplace=True)
print(len(merged_df))

15312


In [37]:
# Define el nuevo orden de las columnas
column_order = ['datetime', 'target', 'temperature', 'dewpoint', 'rain', 'snowfall',
                'surface_pressure', 'cloudcover_low', 'cloudcover_mid', 
                'cloudcover_high', 'windspeed_10m', 'winddirection_10m', 
                'shortwave_radiation', 
                'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh','eic_count', 'installed_capacity']

# Reordena el DataFrame
merged_df = merged_df[column_order]


In [38]:
merged_df.columns

Index(['datetime', 'target', 'temperature', 'dewpoint', 'rain', 'snowfall',
       'surface_pressure', 'cloudcover_low', 'cloudcover_mid',
       'cloudcover_high', 'windspeed_10m', 'winddirection_10m',
       'shortwave_radiation', 'lowest_price_per_mwh', 'highest_price_per_mwh',
       'euros_per_mwh', 'eic_count', 'installed_capacity'],
      dtype='object')

In [39]:
merged_df.head()

,datetime,target,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity
0,2021-09-01 00:00:00,5155.056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-01 01:00:00,4953.258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-01 02:00:00,4910.692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-01 03:00:00,4700.725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-01 04:00:00,4857.681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
merged_df.isnull().sum()

datetime                  0
target                    0
temperature              48
dewpoint                 48
rain                     48
snowfall                 48
surface_pressure         48
cloudcover_low           48
cloudcover_mid           48
cloudcover_high          48
windspeed_10m            48
winddirection_10m        48
shortwave_radiation      48
lowest_price_per_mwh     24
highest_price_per_mwh    24
euros_per_mwh            24
eic_count                72
installed_capacity       72
dtype: int64

## Merged Production

In [41]:
merged_df_p = train_p.merge(historical_weather, left_on=['datetime',], right_on=['date_2_join',], how='left')
merged_df_p.drop(columns=['date_2_join'], inplace=True)
merged_df_p = merged_df_p.merge(gas_prices, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df_p.drop(columns=['date_2_join'], inplace=True)
merged_df_p = merged_df_p.merge(electricity_prices, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df_p.drop(columns=['date_2_join'], inplace=True)
merged_df_p = merged_df_p.merge(client, left_on=['date'], right_on=['date_2_join'], how='left')
merged_df_p.drop(columns=['date_2_join'], inplace=True)

In [42]:
# Define el nuevo orden de las columnas
column_order = ['datetime', 'target', 'temperature', 'dewpoint', 'rain', 'snowfall',
                'surface_pressure', 'cloudcover_low', 'cloudcover_mid', 
                'cloudcover_high', 'windspeed_10m', 'winddirection_10m', 
                'shortwave_radiation', 
                'lowest_price_per_mwh', 'highest_price_per_mwh', 'euros_per_mwh','eic_count', 'installed_capacity']

# Reordena el DataFrame
merged_df_p = merged_df_p[column_order]


In [43]:
merged_df_p.columns

Index(['datetime', 'target', 'temperature', 'dewpoint', 'rain', 'snowfall',
       'surface_pressure', 'cloudcover_low', 'cloudcover_mid',
       'cloudcover_high', 'windspeed_10m', 'winddirection_10m',
       'shortwave_radiation', 'lowest_price_per_mwh', 'highest_price_per_mwh',
       'euros_per_mwh', 'eic_count', 'installed_capacity'],
      dtype='object')

In [44]:
merged_df_p.head()

,datetime,target,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity
0,2021-09-01 00:00:00,0.315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-09-01 01:00:00,0.353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-09-01 02:00:00,0.338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-01 03:00:00,0.303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-09-01 04:00:00,0.424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
merged_df_p.isnull().sum()

datetime                  0
target                    0
temperature              48
dewpoint                 48
rain                     48
snowfall                 48
surface_pressure         48
cloudcover_low           48
cloudcover_mid           48
cloudcover_high          48
windspeed_10m            48
winddirection_10m        48
shortwave_radiation      48
lowest_price_per_mwh     24
highest_price_per_mwh    24
euros_per_mwh            24
eic_count                72
installed_capacity       72
dtype: int64

In [46]:
merged_p_path = root.DIR_DATA_STAGE + 'merged_df_production.pkl'
merged_df_p.to_pickle(merged_p_path) 

In [47]:
merged_c_path = root.DIR_DATA_STAGE + 'merged_df_consumption.pkl'
merged_df.to_pickle(merged_c_path) 

In [40]:
len(merged_df)

15312